<a href="https://colab.research.google.com/github/xiner-hash/feature_engineering-/blob/main/(04.18)%20Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

from numpy import loadtxt

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm

from statistics import *

from xgboost import XGBClassifier

from matplotlib import pyplot

from sklearn import tree, metrics
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


## Time-related

In [101]:
"""
action = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')
affect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')
submission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')
user = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')
user['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))
user['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))
"""

"\naction = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')\naffect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')\nsubmission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')\nuser = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')\nuser['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))\nuser['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))\n"

In [102]:
df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')

In [103]:
# convert action_date column to datetime format
df['action_date'] = pd.to_datetime(df['action_date'])


df['time_of_day'] = pd.cut(df['action_date'].dt.hour,
                           bins=[6, 12, 18, 24],
                           labels=['Morning', 'Afternoon', 'Evening'],
                           include_lowest=False,
                           ordered=False)


# generate Extreme Time of Day feature
df['extreme_time_of_day'] = pd.cut(df['action_date'].dt.hour,
                                   bins=[0, 4, 8, 21, 24],
                                   labels=['late_night', 'early_morning', 'non_extreme_hours', 'late_night'],
                                   ordered=False)

# generate Day of Week feature
df['day_of_week'] = df['action_date'].dt.day_name()
df['day_of_week'] = df['day_of_week'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')


# Create dummy variables
time_of_day_dummies = pd.get_dummies(df['time_of_day'], prefix='time_of_day')
extreme_time_of_day_dummies = pd.get_dummies(df['extreme_time_of_day'], prefix='extreme_time_of_day')
day_of_week_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week')

# concatenate dummy variables with original dataframe
df = pd.concat([df, time_of_day_dummies, extreme_time_of_day_dummies, day_of_week_dummies], axis=1)

In [104]:
df = df.drop(['time_of_day', 'extreme_time_of_day','day_of_week'], axis=1)


In [105]:
df = df.drop(["action_date"], axis=1)

In [106]:
df2 = df

In [107]:
df2

,task_submit_id,user_id,task_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend
0,207153,1582,372,1,1,0,0,1,0,0,1,1,0
1,207157,1582,372,1,1,0,0,1,0,0,1,1,0
2,207173,1573,372,1,1,0,0,0,1,0,0,1,0
3,207178,1573,372,1,0,0,0,0,1,0,0,1,0
4,207180,1573,372,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,347,1,0,0,1,0,0,0,1,0,1
75215,291784,1681,347,1,1,0,1,0,0,0,1,0,1
75216,291786,1681,347,1,0,0,1,0,0,0,1,0,1
75217,291787,1681,347,1,1,0,1,0,0,0,1,0,1


## Percentage Correctness

In [108]:
# df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')


In [109]:
df

,task_submit_id,user_id,task_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend
0,207153,1582,372,1,1,0,0,1,0,0,1,1,0
1,207157,1582,372,1,1,0,0,1,0,0,1,1,0
2,207173,1573,372,1,1,0,0,0,1,0,0,1,0
3,207178,1573,372,1,0,0,0,0,1,0,0,1,0
4,207180,1573,372,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,347,1,0,0,1,0,0,0,1,0,1
75215,291784,1681,347,1,1,0,1,0,0,0,1,0,1
75216,291786,1681,347,1,0,0,1,0,0,0,1,0,1
75217,291787,1681,347,1,1,0,1,0,0,0,1,0,1


In [110]:
# Create new columns for the desired features
df["last_3_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(3).sum().reset_index(level=0, drop=True)
df["last_5_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(5).sum().reset_index(level=0, drop=True)
df["last_8_attempts_incorrect"] = df.groupby("user_id")["test_succeeded"].rolling(8).sum().reset_index(level=0, drop=True)
df["last_3_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(3).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)
df["last_5_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(5).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)
df["last_8_attempts_correct"] = df.groupby("user_id")["test_succeeded"].rolling(8).apply(lambda x: x[::-1].cumsum()[::-1][0], raw=True).reset_index(level=0, drop=True)

# Replace NaN values with 0
df.fillna(0, inplace=True)

In [111]:
#df = pd.merge(df, df2, on='task_submit_id')

In [112]:
df

,task_submit_id,user_id,task_id,test_id,test_succeeded,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct
0,207153,1582,372,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0
1,207157,1582,372,1,1,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207173,1573,372,1,1,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
3,207178,1573,372,1,0,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
4,207180,1573,372,1,1,0,0,0,1,0,0,1,0,2.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75214,291783,1681,347,1,0,0,1,0,0,0,1,0,1,0.0,1.0,2.0,0.0,1.0,2.0
75215,291784,1681,347,1,1,0,1,0,0,0,1,0,1,1.0,2.0,3.0,1.0,2.0,3.0
75216,291786,1681,347,1,0,0,1,0,0,0,1,0,1,1.0,1.0,2.0,1.0,1.0,2.0
75217,291787,1681,347,1,1,0,1,0,0,0,1,0,1,2.0,2.0,3.0,2.0,2.0,3.0


## Attempt Count

In [113]:
# create new columns for the features
df['cumulative_correct'] = 0
df['cumulative_incorrect'] = 0

# initialize variables to keep track of correct and incorrect answers
correct_count = 0
incorrect_count = 0
current_task_id = None

# loop through the rows of the DataFrame
for i, row in df.iterrows():
    # if the user_id changed, reset the counts
    if row['user_id'] != current_task_id:
        correct_count = 0
        incorrect_count = 0
        current_task_id = row['user_id']
    
    # update the counts based on the test_succeeded
    if row['test_succeeded'] == 1:
        correct_count += 1
    else:
        incorrect_count += 1
    
    # update the new columns in the DataFrame
    df.at[i, 'cumulative_correct'] = correct_count
    df.at[i, 'cumulative_incorrect'] = incorrect_count


In [114]:
df = df.drop(["test_id",'test_succeeded',"task_id",'user_id'], axis=1)

## Chunk Data and Aggregate Feature

In [115]:
df2 = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')

In [116]:
df2 = df2.drop(["action_date"], axis=1)


In [117]:
df = pd.merge(df, df2, on='task_submit_id')
df = df.drop(['task_submit_id'], axis=1)


In [119]:
df["task_status"] = 0

df["user_id"] = df["user_id"].astype("string")
df["task_id"] = df["task_id"].astype("string")

df["task_change"] = df["task_id"].shift(1, fill_value=df["task_id"].iloc[0]) != df["task_id"]

df.loc[df["task_change"] == True, "task_status"] = True


df["student_change"] = df["user_id"].shift(1, fill_value=df["user_id"].iloc[0]) != df["user_id"]
df.loc[df["student_change"] == True, "task_status"] = True

df["task_status"].value_counts()


df['new_task_id'] = df['task_status'].cumsum()

df['new_task_id'] = df['new_task_id'].astype(int)
df = df.drop(['task_id', 'task_change', 'student_change','task_status'], axis=1) 

df = df.rename(columns={'new_task_id': 'task_id'})



df = df.reset_index(drop=True)
df["task_id"]=df["task_id"]+1

In [120]:
df

,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect,user_id,affect,task_id
0,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1582,AN,1
1,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,3,2,1573,FO,2
2,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,3,2,1573,OT,2
3,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,4,2,1573,FO,2
4,0,0,0,1,0,0,1,0,2.0,3.0,0.0,2.0,3.0,0.0,4,2,1573,OT,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19259,0,0,1,0,0,1,1,0,0.0,1.0,1.0,0.0,1.0,1.0,1,4,1674,FO,4805
19260,1,0,0,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1809,CO,4806
19261,1,0,0,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,2,1809,CO,4806
19262,1,0,0,0,0,1,0,1,1.0,1.0,1.0,1.0,1.0,1.0,1,2,1809,CO,4806


In [121]:
unique_affect = df['affect'].unique()
print(unique_affect)

affect_counts = df['affect'].value_counts()
print(affect_counts)


['AN' 'FO' 'OT' 'BO' 'CO' 'FR']
FO    9863
FR    2807
OT    2217
AN    1978
CO    1520
BO     879
Name: affect, dtype: int64


In [122]:
df  = df.groupby("task_id").last()


In [124]:
features = df.loc[:, 'time_of_day_Morning':'cumulative_incorrect']

# SumattemptCount (no time) or time_of_day_Morning

features.fillna(features.mean(), inplace=True)
students = df['user_id'].values.tolist()

In [125]:
affect_labels = ['AN','FO','OT','BO','CO','FR']
affect_label = df['affect']

In [ ]:
df

# Model Building 

### **Random Forest**

In [130]:
# 5-fold student-level cross validation 

group_dict = {}
groups = np.array([])
for index, row in df.iterrows():
    student_id = row['user_id']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])
gkf = GroupKFold(n_splits=5)

In [131]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = RandomForestClassifier(n_estimators=100,random_state=1)

        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5000136075077005
Affective label:  FO
AUC ROC performance:  0.5059004506145927
Affective label:  OT
AUC ROC performance:  0.5021978584609675
Affective label:  BO
AUC ROC performance:  0.5101898268406094
Affective label:  CO
AUC ROC performance:  0.5054542213687144
Affective label:  FR
AUC ROC performance:  0.5013543539193916


### **ANN**

In [132]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = MLPClassifier(random_state=1,solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 3))


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5003520706476433
Affective label:  FO
AUC ROC performance:  0.499601593625498
Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.49978189749182117
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.49976648508939814


### **Decision Tree**

In [133]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = DecisionTreeClassifier(random_state=1,max_leaf_nodes= None, min_samples_leaf= 2, min_samples_split=10)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.4991179753717718
Affective label:  FO
AUC ROC performance:  0.5067330076377462
Affective label:  OT
AUC ROC performance:  0.49966189875726663
Affective label:  BO
AUC ROC performance:  0.5034721220519132
Affective label:  CO
AUC ROC performance:  0.4994300978933012
Affective label:  FR
AUC ROC performance:  0.49366892612904223


### **SVM**

In [134]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = svm.SVC(kernel='rbf', decision_function_shape='ovr',random_state = 1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5
Affective label:  FO
AUC ROC performance:  0.531809623078531
Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.5
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.49988385598141694


### **Naive Bayes**

In [135]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = GaussianNB(priors=[0.1, 0.9])


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))




Affective label:  AN
AUC ROC performance:  0.5318941179751586
Affective label:  FO
AUC ROC performance:  0.49738223635600665
Affective label:  OT
AUC ROC performance:  0.4886671771753409
Affective label:  BO
AUC ROC performance:  0.5079289127269463
Affective label:  CO
AUC ROC performance:  0.5278965193000197
Affective label:  FR
AUC ROC performance:  0.5130945239532928


### **XGB**

In [136]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = XGBClassifier(learning_rate=0.05, n_estimators=50, random_state=1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.49988839285714287
Affective label:  FO
AUC ROC performance:  0.5296470694756521
Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.4992366412213741
Affective label:  CO
AUC ROC performance:  0.4997745208568207
Affective label:  FR
AUC ROC performance:  0.49929524842618667
